In [1]:
from smartpool_config import *

spark = create_spark("smartpool-gold-electricity")

print("Spark OK:", spark.version)
print("BASE:", BASE)
print("BRONZE:", BRONZE)
print("SILVER:", SILVER)
print("GOLD:", GOLD)
print("STATE:", STATE)
print("JDBC:", JDBC_URL)


:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/jovyan/.ivy2.5.2/cache
The jars for the packages stored in: /home/jovyan/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-379c133f-8b19-4dc4-a8cb-31d7a0d77025;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 119ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-spark_2.13;4.0.0 from central in [default]
	io.delta#delta-storage;4.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.13.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf  

Spark OK: 4.0.1
BASE: s3a://spark/medallion
BRONZE: s3a://spark/medallion/bronze
SILVER: s3a://spark/medallion/silver
GOLD: s3a://spark/medallion/gold
STATE: s3a://spark/medallion/_state
JDBC: jdbc:sqlserver://sqlserver:1433;databaseName=smartpool;encrypt=true;trustServerCertificate=true;


In [2]:
# SILVER
SILVER_POOLS   = f"{SILVER}/pools_dim"
SILVER_EVENTS  = f"{SILVER}/maintenance_events"
SILVER_ELEC    = f"{SILVER}/electricity_prices"

# GOLD
GOLD_ELEC_DAILY = f"{GOLD}/electricity_daily_stats"
GOLD_ELEC_PEAK  = f"{GOLD}/electricity_peak_hours"
GOLD_EVENTS_ENR = f"{GOLD}/maintenance_events_enriched"
GOLD_EVENTS_COST= f"{GOLD}/maintenance_events_cost"

pools_s  = spark.read.format("delta").load(SILVER_POOLS)
events_s = spark.read.format("delta").load(SILVER_EVENTS)
elec_s   = spark.read.format("delta").load(SILVER_ELEC)

print("Silver pools :", pools_s.count())
print("Silver events:", events_s.count())
print("Silver elec  :", elec_s.count())

display(elec_s.orderBy(F.col("date").desc(), F.col("hour").asc()).limit(10))
display(events_s.orderBy(F.col("event_time").desc()).limit(10))


26/01/25 22:41:13 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
26/01/25 22:41:18 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Silver pools : 6


Silver events: 16
Silver elec  : 720


DataFrame[ts_utc: string, hour: int, price_eur_mwh: double, price_eur_kwh: double, region: string, source: string, source_file: string, ingest_ts: timestamp, ingest_date: date, date: date, silver_ingest_ts: timestamp]

DataFrame[pool_id: int, id: int, event_time: timestamp, intervention_type: string, product_type: string, product_amount: double, notes: string, updated_at: timestamp, event_date: date]

In [3]:
gold_elec_daily = (elec_s
    .groupBy("date", "region")
    .agg(
        F.avg("price_eur_kwh").alias("avg_price_eur_kwh"),
        F.min("price_eur_kwh").alias("min_price_eur_kwh"),
        F.max("price_eur_kwh").alias("max_price_eur_kwh"),
        F.avg("price_eur_mwh").alias("avg_price_eur_mwh"),
        F.min("price_eur_mwh").alias("min_price_eur_mwh"),
        F.max("price_eur_mwh").alias("max_price_eur_mwh"),
        F.count("*").alias("rows")
    )
    .withColumn("gold_calc_ts", F.current_timestamp())
)

(gold_elec_daily.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .partitionBy("date")
    .save(GOLD_ELEC_DAILY)
)

print("GOLD electricity_daily_stats OK ->", GOLD_ELEC_DAILY, "rows:", gold_elec_daily.count())
display(gold_elec_daily.orderBy(F.col("date").desc(), F.col("region")))


GOLD electricity_daily_stats OK -> s3a://spark/medallion/gold/electricity_daily_stats rows: 30


DataFrame[date: date, region: string, avg_price_eur_kwh: double, min_price_eur_kwh: double, max_price_eur_kwh: double, avg_price_eur_mwh: double, min_price_eur_mwh: double, max_price_eur_mwh: double, rows: bigint, gold_calc_ts: timestamp]

In [4]:
TOP_N = 5

w = Window.partitionBy("date", "region").orderBy(F.col("price_eur_kwh").desc(), F.col("hour").asc())

gold_peak = (elec_s
    .select("date", "region", "hour", "price_eur_kwh", "price_eur_mwh")
    .withColumn("rank", F.row_number().over(w))
    .filter(F.col("rank") <= F.lit(TOP_N))
    .withColumn("gold_calc_ts", F.current_timestamp())
)

(gold_peak.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .partitionBy("date")
    .save(GOLD_ELEC_PEAK)
)

print("GOLD electricity_peak_hours OK ->", GOLD_ELEC_PEAK, "rows:", gold_peak.count())
display(gold_peak.orderBy(F.col("date").desc(), F.col("region"), F.col("rank")))


GOLD electricity_peak_hours OK -> s3a://spark/medallion/gold/electricity_peak_hours rows: 150


DataFrame[date: date, region: string, hour: int, price_eur_kwh: double, price_eur_mwh: double, rank: int, gold_calc_ts: timestamp]

In [5]:
# Prepara events con date/hour
events_enrich_base = (events_s
    .withColumn("event_date", F.to_date("event_time"))
    .withColumn("event_hour", F.hour("event_time"))
)

# Prepara elec (keys de join)
elec_key = (elec_s
    .select(
        F.col("date").alias("event_date"),
        F.col("hour").alias("event_hour"),
        "region",
        "price_eur_kwh",
        "price_eur_mwh"
    )
)

# Como solo tenemos ESPAÑA, con esto basta. Si en el futuro hay más regiones, derivariamos por location.
DEFAULT_REGION = "ES"

gold_events_enriched = (events_enrich_base
    .withColumn("region", F.lit(DEFAULT_REGION))
    .join(elec_key, on=["event_date", "event_hour", "region"], how="left")
    .join(pools_s.select("pool_id","pool_name","location","volume_liters","is_heated","owner_type"), on="pool_id", how="left")
    .withColumn("gold_calc_ts", F.current_timestamp())
)

(gold_events_enriched.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .partitionBy("event_date")
    .save(GOLD_EVENTS_ENR)
)

print("GOLD maintenance_events_enriched OK ->", GOLD_EVENTS_ENR, "rows:", gold_events_enriched.count())
display(gold_events_enriched.orderBy(F.col("event_time").desc()).limit(30))


GOLD maintenance_events_enriched OK -> s3a://spark/medallion/gold/maintenance_events_enriched rows: 16


DataFrame[pool_id: int, event_date: date, event_hour: int, region: string, id: int, event_time: timestamp, intervention_type: string, product_type: string, product_amount: double, notes: string, updated_at: timestamp, price_eur_kwh: double, price_eur_mwh: double, pool_name: string, location: string, volume_liters: int, is_heated: boolean, owner_type: string, gold_calc_ts: timestamp]

In [6]:
# Consumo (kWh) por tipo de intervención
energy_map = F.create_map(
    F.lit("filter_backwash"), F.lit(1.2),
    F.lit("refill"),          F.lit(0.8),
    F.lit("chlorine"),        F.lit(0.0),
    F.lit("ph_correction"),   F.lit(0.0),
)

gold_events_cost = (gold_events_enriched
    .withColumn("estimated_energy_kwh", energy_map[F.col("intervention_type")])
    .withColumn("estimated_cost_eur", F.col("estimated_energy_kwh") * F.col("price_eur_kwh"))
    .withColumn("estimated_cost_eur", F.round(F.col("estimated_cost_eur"), 4))
)

(gold_events_cost.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .partitionBy("event_date")
    .save(GOLD_EVENTS_COST)
)

print("GOLD maintenance_events_cost OK ->", GOLD_EVENTS_COST, "rows:", gold_events_cost.count())
display(gold_events_cost.orderBy(F.col("event_time").desc()).limit(30))


GOLD maintenance_events_cost OK -> s3a://spark/medallion/gold/maintenance_events_cost rows: 16


DataFrame[pool_id: int, event_date: date, event_hour: int, region: string, id: int, event_time: timestamp, intervention_type: string, product_type: string, product_amount: double, notes: string, updated_at: timestamp, price_eur_kwh: double, price_eur_mwh: double, pool_name: string, location: string, volume_liters: int, is_heated: boolean, owner_type: string, gold_calc_ts: timestamp, estimated_energy_kwh: double, estimated_cost_eur: double]

In [7]:
g_daily = spark.read.format("delta").load(GOLD_ELEC_DAILY)
g_peak  = spark.read.format("delta").load(GOLD_ELEC_PEAK)
g_enr   = spark.read.format("delta").load(GOLD_EVENTS_ENR)
g_cost  = spark.read.format("delta").load(GOLD_EVENTS_COST)

print("GOLD daily rows:", g_daily.count())
print("GOLD peak  rows:", g_peak.count())
print("GOLD enr   rows:", g_enr.count())
print("GOLD cost  rows:", g_cost.count())

# Check: peaks = TOP_N por date y region (si hay datos completos)
g_peak.groupBy("date","region").agg(F.count("*").alias("rows")).orderBy(F.col("date").desc()).show(20, truncate=False)

# Check: enriched debería tener price_eur_kwh no nulo salvo que falte ese día/hora
g_enr.select(
    F.count("*").alias("rows"),
    F.sum(F.when(F.col("price_eur_kwh").isNull(), 1).otherwise(0)).alias("rows_without_price")
).show(truncate=False)

# Métrica vistosa: coste estimado por piscina
(g_cost.groupBy("pool_id", "pool_name")
    .agg(
        F.count("*").alias("num_events"),
        F.sum("estimated_cost_eur").alias("sum_est_cost_eur"),
        F.avg("estimated_cost_eur").alias("avg_est_cost_eur")
    )
    .orderBy(F.col("sum_est_cost_eur").desc())
).show(20, truncate=False)

print("QA FINAL OK")


GOLD daily rows: 30
GOLD peak  rows: 150
GOLD enr   rows: 16
GOLD cost  rows: 16
+----------+------+----+
|date      |region|rows|
+----------+------+----+
|2026-02-13|ES    |5   |
|2026-02-12|ES    |5   |
|2026-02-11|ES    |5   |
|2026-02-10|ES    |5   |
|2026-02-09|ES    |5   |
|2026-02-08|ES    |5   |
|2026-02-07|ES    |5   |
|2026-02-06|ES    |5   |
|2026-02-05|ES    |5   |
|2026-02-04|ES    |5   |
|2026-02-03|ES    |5   |
|2026-02-02|ES    |5   |
|2026-02-01|ES    |5   |
|2026-01-31|ES    |5   |
|2026-01-30|ES    |5   |
|2026-01-29|ES    |5   |
|2026-01-28|ES    |5   |
|2026-01-27|ES    |5   |
|2026-01-26|ES    |5   |
|2026-01-25|ES    |5   |
+----------+------+----+
only showing top 20 rows
+----+------------------+
|rows|rows_without_price|
+----+------------------+
|16  |0                 |
+----+------------------+

+-------+-------------------------------+----------+----------------+--------------------+
|pool_id|pool_name                      |num_events|sum_est_cost_eur|avg

In [8]:
(g_enr
  .groupBy("event_date")
  .agg(
      F.count("*").alias("events"),
      F.sum(F.when(F.col("price_eur_kwh").isNull(), 1).otherwise(0)).alias("events_without_price"),
      (F.count("*") - F.sum(F.when(F.col("price_eur_kwh").isNull(), 1).otherwise(0))).alias("events_with_price")
  )
  .orderBy(F.col("event_date").desc())
).show(50, truncate=False)


+----------+------+--------------------+-----------------+
|event_date|events|events_without_price|events_with_price|
+----------+------+--------------------+-----------------+
|2026-01-25|2     |0                   |2                |
|2026-01-24|2     |0                   |2                |
|2026-01-23|2     |0                   |2                |
|2026-01-22|2     |0                   |2                |
|2026-01-21|1     |0                   |1                |
|2026-01-20|1     |0                   |1                |
|2026-01-19|1     |0                   |1                |
|2026-01-18|2     |0                   |2                |
|2026-01-17|1     |0                   |1                |
|2026-01-16|1     |0                   |1                |
|2026-01-15|1     |0                   |1                |
+----------+------+--------------------+-----------------+



In [9]:
spark.stop()